In [ ]:
import cv2
import os
from datetime import datetime
import face_recognition
import numpy as np
import pandas as pd
import pickle
import serial

class MultiFacerec:
    def __init__(self):
        # Define the photos folder path
        self.photos_folder = os.path.join(os.path.expanduser("~"), "Downloads", "Photos")

        # Other initialization code...
        self.model = 'face_recognition'
        self.frame_resizing = 0.5
        self.faces_folder = os.path.join(os.path.expanduser("~"), "Downloads", "Faces")
        self.intrusion_folder = os.path.join(os.path.expanduser("~"), "Downloads", "Intrusion")
        self.csv_file_path = os.path.join(os.path.expanduser("~"), "Downloads", "recognized_faces.csv")
        self.presentees = set()
        self.unique_entries = set()
        self.unique_names = set()
        self.confidence_tolerance = 0.5  # Adjust confidence tolerance
        self.training_images_per_person = 5  # Number of training images per person

        # Lists to store known face encodings and names
        self.known_face_encodings = []
        self.known_face_names = []

        # Initialize serial communication with Arduino
        self.arduino_port = '/dev/cu.usbmodem14101'  # Modify port as needed
        self.arduino_baudrate = 9600
        self.ser = serial.Serial(self.arduino_port, self.arduino_baudrate)

        # Flag to indicate if Ashish's face has been recognized
        self.ashish_recognized = False

        # Load or download face encodings
        self.load_or_download_encodings()

        # Buffer to store recognized faces
        self.recognized_faces_buffer = []

    def load_or_download_encodings(self):
        encoding_file_path = os.path.join(os.path.expanduser("~"), "Downloads", "face_encodings.pkl")

        # If encoding file exists, load it; otherwise, download and store it
        if os.path.exists(encoding_file_path):
            with open(encoding_file_path, 'rb') as file:
                encoding_data = pickle.load(file)
                self.known_face_encodings = encoding_data["encodings"]
                self.known_face_names = encoding_data["names"]
                print("Face encodings loaded from file.")
        else:
            self.download_and_store_encodings(encoding_file_path)

    def download_and_store_encodings(self, encoding_file_path):
        for filename in os.listdir(self.faces_folder):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                name, _ = os.path.splitext(filename)

                img_path = os.path.join(self.faces_folder, filename)
                img = cv2.imread(img_path)
                rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_encodings = face_recognition.face_encodings(rgb_img)

                # Use only the first `training_images_per_person` images for training
                for i in range(min(len(img_encodings), self.training_images_per_person)):
                    img_encoding = img_encodings[i]
                    self.known_face_encodings.append(img_encoding)
                    self.known_face_names.append({"name": name})
                    print(f"Encoding image {name} loaded")

        encoding_data = {"encodings": self.known_face_encodings, "names": self.known_face_names}
        with open(encoding_file_path, 'wb') as file:
            pickle.dump(encoding_data, file)
            print("Face encodings saved to file.")

    def mark_attendance(self, name, timestamp):
        if name == "Unknown" or name in self.presentees or name in self.unique_entries:
            return

        self.presentees.add(name)
        self.unique_entries.add(name)
        self.unique_names.add(name)

    def sort_and_write_csv(self, file_path, data):
        if not data:
            print("No recognized faces to append.")
            return

        df = pd.DataFrame(data)

        if os.path.exists(file_path):
            existing_df = pd.read_csv(file_path)
            new_df = pd.concat([existing_df, df], ignore_index=True)
            new_df.drop_duplicates(inplace=True)  # Remove duplicates
            new_df.to_csv(file_path, index=False)
            print("Recognized faces appended to CSV.")
            print("File created successfully")  # Only print if recognized faces are appended
        else:
            df.to_csv(file_path, index=False)

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for face_encoding, face_loc in zip(face_encodings, face_locations):
            self.process_face(frame, face_encoding, face_loc)

    def process_face(self, frame, face_encoding, face_loc):
        face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)

        # Find the best match with confidence tolerance
        best_match_index = np.argmin(face_distances)
        min_face_distance = face_distances[best_match_index]

        if min_face_distance <= self.confidence_tolerance:
            # Face is recognized
            entry = self.known_face_names[best_match_index]
            name = entry["name"]

            if name not in self.presentees:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                entry["timestamp"] = timestamp
                print(f"{name} recognized at {timestamp}")
                self.presentees.add(name)

                self.mark_attendance(name, timestamp)
                self.recognized_faces_buffer.append({"name": name, "timestamp": timestamp})  # Append recognized face data

                # Adjust bounding box coordinates to ensure the entire face is captured
                top, right, bottom, left = face_loc
                face_height = bottom - top
                face_width = right - left
                face_margin = 3  # Increase or decrease this value as needed to adjust the margin around the face
                top = max(0, int(top - face_height * face_margin))
                bottom = min(frame.shape[0], int(bottom + face_height * face_margin))
                left = max(0, int(left - face_width * face_margin))
                right = min(frame.shape[1], int(right + face_width * face_margin))

                # Save the whole face region as an image
                face_image = frame[top:bottom, left:right]
                face_image_path = os.path.join(self.photos_folder, f"{name}_{timestamp}.jpg")
                cv2.imwrite(face_image_path, face_image)
                print(f"Face image saved: {face_image_path}")

        else:
            # Unknown face (intruder)
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            unknown_face_image_path = os.path.join(self.intrusion_folder, f"Intrusion_{timestamp}.jpg")

            # Adjust bounding box coordinates to ensure the entire face is captured
            top, right, bottom, left = face_loc
            face_height = bottom - top
            face_width = right - left
            face_margin = 3  # Increase or decrease this value as needed to adjust the margin around the face
            top = max(0, int(top - face_height * face_margin))
            bottom = min(frame.shape[0], int(bottom + face_height * face_margin))
            left = max(0, int(left - face_width * face_margin))
            right = min(frame.shape[1], int(right + face_width * face_margin))

            # Save the whole face region as an image
            unknown_face_image = frame[top:bottom, left:right]
            cv2.imwrite(unknown_face_image_path, unknown_face_image)
            print(f"Unknown face image saved: {unknown_face_image_path}")
            self.presentees.add("Unknown")  # Mark the intruder as present to prevent saving more unknown faces

    def send_code_to_arduino(self, code):
        try:
            self.ser.write(str(code).encode())
            print(f"Code '{code}' sent to Arduino")
        except Exception as e:
            print(f"Error sending code to Arduino: {e}")

# Initialize recognized_faces list
recognized_faces = []

# Create an instance of MultiFacerec with the desired model
mfr = MultiFacerec()

# Initialize the camera
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Detect faces and process
    mfr.detect_known_faces(frame)

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Update CSV file
        mfr.sort_and_write_csv(mfr.csv_file_path, recognized_faces)

        # Sending code to Arduino based on recogniized person
        for entry in mfr.recognized_faces_buffer:
            name = entry["name"]
            if name == "Person A":
                print("Sending 1 to Arduino...")
                mfr.send_code_to_arduino(1)
            elif name == "Person B":
                print("Sending 2 to Arduino...")
                mfr.send_code_to_arduino(2)
            elif name == "Person C":
                print("Sending 3 to Arduino...")
                mfr.send_code_to_arduino(3)
            elif name == "Person D":
                print("Sending 4 to Arduino...")
                mfr.send_code_to_arduino(4)

        print("Quitting...")
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
